# Этот скрипт позволяет поменять процентное соотношение по частотам
Что можно задавать самим?
- сайты, по которым будут изменения
- проценты на частотах 6-9, 10-99 и 100+
- разброс показов в рамках частот 10-99 и 6-9
- в каком процентном соотношении переносятся показы на частоты 2 и 3

In [55]:
import pandas as pd
import numpy as np
import random

In [57]:
#функция для того, чтобы у нас был небольшой разброс частот в рамках 10-99 и 6-9

def controlled_random_split(total, n, deviation_ratio=0.1):
    """
    Делит total на n целых чисел с ограниченным разбросом.
    deviation_ratio - максимальное отклонение от среднего, например 0.1 = ±10%
    """
    base_value = total // n
    remaining = total - base_value * n

    # Генерируем отклонения
    deviations = [random.uniform(-deviation_ratio, deviation_ratio) for _ in range(n)]
    
    # Применяем отклонения
    adjusted_values = [max(int(base_value * (1 + dev)), 0) for dev in deviations]

    # Корректируем сумму (если разброс чуть уводит итог)
    adjusted_sum = sum(adjusted_values)
    diff = total - adjusted_sum

    # Раздаем остаток по одной единице, чтобы точно совпало
    for i in range(abs(diff)):
        idx = random.randint(0, n - 1)
        if diff > 0:
            adjusted_values[idx] += 1
        elif diff < 0 and adjusted_values[idx] > 0:
            adjusted_values[idx] -= 1

    return adjusted_values

### В этом блоке нам нужно будет выбрать номера файлов, плейсменты которых мы будем править

In [62]:
# Чтение Excel файла
df = pd.read_excel('frequency_by_placements_1000195072_2025-06-01_2025-06-15_ver_40b6ea813ee7.xlsx')

# Выбор Site Name
unique_sites = df['Site Name'].unique()
print("\nДоступные Site Name:")
for i, site in enumerate(unique_sites):
    print(f"{i + 1}. {site}")

selected_indices = input("\nВведите номера Site Name, которые будем редактировать (через запятую): ")
selected_indices = [int(i.strip()) - 1 for i in selected_indices.split(',')]
selected_sites = [unique_sites[i] for i in selected_indices]

# Приведение Frequency к числовому типу
df['Frequency'] = df['Frequency'].replace('100+', 100)
df['Frequency'] = pd.to_numeric(df['Frequency'], errors='coerce')

df_copy = df.copy()


Доступные Site Name:
1. IVI.ru
2. Avito.ru
3. VKontakte (RU)
4. Mediasniper.ru
5. DSP Yabbi RU
6. MediaDesk RU
7. MTS.ru
8. Hyper RU
9. Solta
10. Ekran.ru
11. Astralab RU
12. Buzzoola.com
13. Trading Desk RU
14. GPM RU
15. Plazkart RU
16. Digital Alliance RU
17. TechDig RU
18. Digital Max RU
19. Mobidriven
20. Innovation Lab RU



Введите номера Site Name, которые будем редактировать (через запятую):  18


/var/folders/t1/5tdq7kw91qldw83rpf44n3fr0000gn/T/ipykernel_86950/364422257.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Frequency'] = df['Frequency'].replace('100+', 100)


### В этом блоке мы можем поменять проценты (блок # Диапазоны частот и процентов)
Процент задается рандомно от минимально указанного до максимально указанного для того, чтобы в разных плейсментах у нас были немного разные проценты

По дефолту скрипт раскидывает 75% убранных показов на 2 частоту, 25% на 3, это соотношение также можно поменять

In [64]:
# Функция для случайного разбиения суммы на n целых чисел
def random_integer_split(total, n):
    if n == 1:
        return [total]
    cuts = sorted(random.sample(range(1, total), n - 1))
    return [a - b for a, b in zip(cuts + [total], [0] + cuts)]

for site in selected_sites:
    site_df = df_copy[df_copy['Site Name'] == site]

    for placement in site_df['Placement Name'].unique():
        placement_mask = (df_copy['Site Name'] == site) & (df_copy['Placement Name'] == placement)
        total_impressions = df_copy.loc[placement_mask, 'Impressions'].sum()

        total_removed = 0

        # Диапазоны частот и процентов
        frequency_groups = [
            {'range': (6, 9), 'min_pct': 0.01, 'max_pct': 0.013},
            {'range': (10, 99), 'min_pct': 0.003, 'max_pct': 0.004},
            {'range': (100, 100), 'min_pct': 0.001, 'max_pct': 0.003}
        ]

        for group in frequency_groups:
            freq_mask = placement_mask & df_copy['Frequency'].between(group['range'][0], group['range'][1])

            if freq_mask.sum() == 0:
                continue

            group_indices = df_copy.loc[freq_mask].index.tolist()
            group_total = df_copy.loc[freq_mask, 'Impressions'].sum()

            # Целевая сумма для группы
            min_target = int(group['min_pct'] * total_impressions)
            max_target = int(group['max_pct'] * total_impressions)
            if max_target < min_target:
                max_target = min_target  # Безопасность

            target_sum = random.randint(min_target, max_target)

            if target_sum >= group_total:
                # Если текущие показы уже меньше целевого — пропускаем
                continue

            # Рандомно разбиваем target_sum на количество строк в группе, если хотим уменьшить или увеличить разброс меняем deviation_ratio
            new_values = controlled_random_split(target_sum, len(group_indices), deviation_ratio=0.1)

            # Сколько показы сократили
            reduced = group_total - sum(new_values)
            total_removed += reduced

            # Обновляем Impressions в df_copy
            for idx, new_val in zip(group_indices, new_values):
                df_copy.at[idx, 'Impressions'] = new_val

        # Перераспределяем убранные показы на частоты 2 и 3
        if total_removed > 0:
            freq2_mask = placement_mask & (df_copy['Frequency'] == 2)
            freq3_mask = placement_mask & (df_copy['Frequency'] == 3)

            freq2_add = round(total_removed * 0.75)
            freq3_add = total_removed - freq2_add

            if df_copy.loc[freq2_mask].shape[0] > 0:
                df_copy.loc[freq2_mask, 'Impressions'] += freq2_add
            else:
                new_row = df_copy[placement_mask].iloc[0].copy()
                new_row['Frequency'] = 2
                new_row['Impressions'] = freq2_add
                df_copy = pd.concat([df_copy, pd.DataFrame([new_row])], ignore_index=True)

            if df_copy.loc[freq3_mask].shape[0] > 0:
                df_copy.loc[freq3_mask, 'Impressions'] += freq3_add
            else:
                new_row = df_copy[placement_mask].iloc[0].copy()
                new_row['Frequency'] = 3
                new_row['Impressions'] = freq3_add
                df_copy = pd.concat([df_copy, pd.DataFrame([new_row])], ignore_index=True)

In [66]:
df_copy.to_excel('res.xlsx', index=False)

### Этот скрипт сравнивает выгрузки за прошлую неделю и текующую и проверяет, есть ли в выгрузке за текущую неделю значения меньше, чем за прошлую

In [10]:
import pandas as pd

# Читаем файлы с учетом правильного разделителя
df_prev = pd.read_excel('old.xlsx')
df_new = pd.read_excel('new.xlsx')

# Убедимся, что нужные колонки есть
required_columns = ['Site Name', 'Placement Name', 'Frequency', 'Impressions']

for col in required_columns:
    if col not in df_prev.columns or col not in df_new.columns:
        raise KeyError(f"Колонка '{col}' не найдена в одном из файлов. Проверь названия колонок.")

# Обработка значений Frequency
for df in [df_prev, df_new]:
    df['Frequency'] = df['Frequency'].replace('100+', 100)
    df['Frequency'] = pd.to_numeric(df['Frequency'], errors='coerce')

# Оставляем только Placement Name, которые есть в обоих файлах
common_placements = set(df_prev['Placement Name']) & set(df_new['Placement Name'])

# Фильтруем файлы
df_prev = df_prev[df_prev['Placement Name'].isin(common_placements)]
df_new = df_new[df_new['Placement Name'].isin(common_placements)]

# Сравнение строк
issues = []

for index, row in df_prev.iterrows():
    site = row['Site Name']
    placement = row['Placement Name']
    frequency = row['Frequency']
    prev_impressions = row['Impressions']

    # Фильтр для новой недели
    match = df_new[
        (df_new['Site Name'] == site) &
        (df_new['Placement Name'] == placement) &
        (df_new['Frequency'] == frequency)
    ]

    if match.empty:
        # Если такой строки нет — пропускаем, так как этот Placement Name мы уже проверили, что он есть
        continue
    else:
        new_impressions = match['Impressions'].sum()
        if new_impressions < prev_impressions:
            issues.append(f"МЕНЬШЕ ПОКАЗОВ: {site} | {placement} | Frequency {frequency} | Предыдущие показы: {prev_impressions} | Новые показы: {new_impressions}")

# Вывод результата в консоль
if issues:
    print("\n⚠️ Найдены строки, где в новой неделе меньше показов, чем в прошлой:\n")
    for issue in issues:
        print(issue)
else:
    print("\n✅ Все ок! Во всех строках количество показов в новой неделе не меньше, чем в прошлой.")

/var/folders/t1/5tdq7kw91qldw83rpf44n3fr0000gn/T/ipykernel_98524/3831090887.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Frequency'] = df['Frequency'].replace('100+', 100)



⚠️ Найдены строки, где в новой неделе меньше показов, чем в прошлой:

МЕНЬШЕ ПОКАЗОВ: Digital Max RU | Digital_Max_OLV_All_25-45_BC_Custom_Segments_OLV_20_sec_desk_CN_apr-jun_2025 | Frequency 6 | Предыдущие показы: 116 | Новые показы: 0
МЕНЬШЕ ПОКАЗОВ: Digital Max RU | Digital_Max_OLV_All_25-45_BC_Custom_Segments_OLV_20_sec_desk_CN_apr-jun_2025 | Frequency 7 | Предыдущие показы: 92 | Новые показы: 0
МЕНЬШЕ ПОКАЗОВ: Digital Max RU | Digital_Max_OLV_All_25-45_BC_Custom_Segments_OLV_20_sec_desk_CN_apr-jun_2025 | Frequency 8 | Предыдущие показы: 73 | Новые показы: 0
МЕНЬШЕ ПОКАЗОВ: Digital Max RU | Digital_Max_OLV_All_25-45_BC_Custom_Segments_OLV_20_sec_desk_CN_apr-jun_2025 | Frequency 9 | Предыдущие показы: 60 | Новые показы: 0
МЕНЬШЕ ПОКАЗОВ: Digital Max RU | Digital_Max_OLV_All_25-45_BC_Custom_Segments_OLV_20_sec_desk_CN_apr-jun_2025 | Frequency 10 | Предыдущие показы: 21 | Новые показы: 0
МЕНЬШЕ ПОКАЗОВ: Digital Max RU | Digital_Max_OLV_All_25-45_BC_Custom_Segments_OLV_20_sec_desk_CN_